In [1]:
%pylab inline
import psycopg2
from pandas.io.sql import read_sql

pgconn = psycopg2.connect(database='asos', host='localhost', port=5555, user='nobody')

df = None
# No events found prior to 1971
for year in range(1971, 2016):
    print year
    df2 = read_sql("""SELECT station, valid at time zone 'UTC', metar from t"""+str(year)+"""
  where length(station) = 3 and length(metar) > 2 and (metar ~* 'TORNADO' or metar ~* '\+FC') ORDER by valid ASC""", pgconn)
    if len(df2) > 0:
        print year, len(df2)
        if df is None:
            df = df2
        else:
            df = df.append(df2)


Populating the interactive namespace from numpy and matplotlib
1971
1971 6
1972
1972 4
1973
1973 279
1974
1974 196
1975
1975 215
1976
1976 121
1977
1977 80
1978
1978 78
1979
1979 113
1980
1980 52
1981
1981 39
1982
1982 40
1983
1983 44
1984
1984 46
1985
1985 40
1986
1986 37
1987
1987 37
1988
1988 52
1989
1989 47
1990
1990 54
1991
1991 85
1992
1992 177
1993
1993 90
1994
1994 128
1995
1995 175
1996
1996 219
1997
1997 184
1998
1998 284
1999
1999 222
2000
2000 118
2001
2001 114
2002
2002 121
2003
2003 222
2004
2004 217
2005
2005 101
2006
2006 154
2007
2007 53
2008
2008 38
2009
2009 44
2010
2010 83
2011
2011 92
2012
2012 70
2013
2013 81
2014
2014 210
2015
2015 68


In [4]:
df.rename(columns={'timezone': 'valid'}, inplace=True)
df.to_excel('151113_metar_tor_reports.xlsx', index=False)


In [14]:
import pytz
df['valid'] = df['valid'].apply(lambda x: x.replace(tzinfo=pytz.timezone("UTC")))

In [23]:
import datetime
cursor = pgconn.cursor()
cursor.execute("SET TIME ZONE 'UTC'")
done = {}
for i, row in df.iterrows():
    key = "%s_%s" % (row['station'], row['valid'].date())
    if key in done:
        continue
    done[key] = True
    cursor.execute("""SELECT valid, metar, valid - %s from t"""+str(row['valid'].year)+"""
    where valid > %s and valid < %s
    and station = %s and metar ~* 'SN ' ORDER by valid ASC""", (row['valid'], row['valid'],
                                            row['valid'] + datetime.timedelta(hours=24),
                                            row['station']))
    if cursor.rowcount > 0:
        row2 = cursor.fetchone()
        cursor.execute("""SELECT name, state, tzname from stations
        where id = %s and network ~* 'ASOS'
        """, (row['station'], ))
        meta = cursor.fetchone()
        localtz = pytz.timezone(meta[2])
        tornts = row['valid'].astimezone(localtz)
        snowts = row2[0].astimezone(localtz)
        print '----------------------------'
        print "ID: %s Date: %s Name: %s State: %s Interval: %.0fh %.0fm\n%s => %s\n%s => %s" % (
            row['station'], tornts.strftime("%d %b %Y"), meta[0], meta[1], row2[2].seconds / 3600, row2[2].seconds % 3600 / 60,
                                             tornts.strftime("%d %b %Y %I:%M %p"),
        row['metar'].strip(),
                                        snowts.strftime("%d %b %Y %I:%M %p"), row2[1].strip())
        

----------------------------
ID: POB Date: 03 Mar 1971 Name: POPE_AFB State: NC Interval: 11h 0m
03 Mar 1971 03:00 PM => METAR KPOB 032000Z 07004KT 15/8SM -RA FG CLR 13/12 A2940 RMK F2 CIG 2V5 PIREP CONFIROMED TORNADO 20E FAY MOVMT UNKN WET RWY 1015 SLPNO 8/935 T01280117
04 Mar 1971 02:00 AM => METAR KPOB 040700Z 30006KT 1SM -SN CLR 02/M02 A2949 RMK R22VR60+ S6 CIG 14V16 PRESRR WET RWY SLPNO 8/015 T00171022
----------------------------
ID: PRC Date: 20 Apr 1973 Name: PRESCOTT/LOVE FLD State: AZ Interval: 7h 0m
20 Apr 1973 01:00 PM => KPRC 202000Z 21010KT 40SM BKN030 09/M08 A2977 RMK RWU ALQDS MTNS OBSCD S-W/UUA TORNADO SIGHTED 10SW P14 B52.20N INW MDT RIME ICG 130 BE36PRC4/18 SLP064 8/950 T00881077
20 Apr 1973 08:00 PM => KPRC 210300Z 12008KT 10SM -SHSN OVC013 01/M03 A2993 RMK RE09 SB10 SLP138 T00111033 52034
----------------------------
ID: MSN Date: 09 May 1973 Name: MADISON State: WI Interval: 1h 0m
09 May 1973 02:00 PM => METAR KMSN 091900Z 22018G27KT 15SM BKN030 OVC042 19/13 A2959